In [ ]:
import pandas as pd
import mftool

In [ ]:
basic_data = pd.read_csv('mf_app/data/mutual_funds.csv')
basic_data.drop(columns=['Unnamed: 0', 'ISIN Div Payout/ ISIN Growth','ISIN Div Reinvestment'], axis=0, inplace=True)

In [ ]:
basic_data.columns = (
    basic_data.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

In [ ]:
direct_growth_mask = (
    basic_data["scheme_name"].str.contains("direct", case=False, na=False) &
    basic_data["scheme_name"].str.contains("growth", case=False, na=False)
)

In [ ]:
string_only_mask = (
    basic_data["scheme_name"].notna() &
    basic_data[["scheme_code", "net_asset_value", "date"]].isna().all(axis=1)
)
final_df = basic_data[direct_growth_mask | string_only_mask].reset_index(drop=True)


In [ ]:
final_df.to_csv("direct_growth_with_headers.csv", index=False)

In [ ]:
from datetime import datetime

def extract_plan(scheme_name: str) -> str:
    return "Direct" if "direct" in scheme_name.lower() else "Regular"


def extract_eq_or_dt(scheme_category: str) -> str:
    if "debt" in scheme_category.lower():
        return "Debt Scheme"
    if "equity" in scheme_category.lower():
        return "Equity Scheme"
    return "Other"


def extract_type_of_mf(scheme_category: str) -> str:
    if "-" in scheme_category:
        return scheme_category.split("-", 1)[1].strip()
    return scheme_category.strip()


In [ ]:
import time
import json
from requests.exceptions import RequestException

def fetch_scheme_meta(mf, scheme_code, retries=3, sleep_sec=0.5):
    for attempt in range(1, retries + 1):
        try:
            return json.loads(mf.get_scheme_historical_nav(
                str(scheme_code),
                as_json=True
            ))
        except RequestException as e:
            if attempt == retries:
                raise
            time.sleep(sleep_sec * attempt)

In [ ]:
import pandas as pd
import json
from datetime import datetime, timezone
from mftool import Mftool
from tqdm import tqdm

mf = Mftool()

processed_codes = set()

rows = []

scheme_codes = (
    pd.to_numeric(final_df["scheme_code"], errors="coerce")
      .dropna()
      .astype(int)
      .unique()
)

for scheme_code in tqdm(scheme_codes):
    if scheme_code in processed_codes:
        continue

    try:
        api_data = fetch_scheme_meta(mf, scheme_code)
    except Exception as e:
        print(f"Skipping {scheme_code}: {e}")
        continue

    row = {
        "scheme_code": api_data["scheme_code"],
        "scheme_name": api_data["scheme_name"],
        "amc": api_data["fund_house"],
        "plan": extract_plan(api_data["scheme_name"]),
        "risk_profile": "Very High",
        "created_at": datetime.now(timezone.utc),
        "started_date": api_data["scheme_start_date"]["date"],
        "eq_or_dt": extract_eq_or_dt(api_data["scheme_category"]),
        "type_of_mf": extract_type_of_mf(api_data["scheme_category"]),
    }

    rows.append(row)
    processed_codes.add(scheme_code)

    # polite throttling
    time.sleep(0.3)


In [ ]:
if len(rows) % 50 == 0:
    pd.DataFrame(rows).to_csv(
        "scheme_master_partial.csv",
        index=False
    )
else:
    pd.DataFrame(rows).to_csv(
        "scheme_master_full.csv",
        index=False
    )

In [2]:
import requests
import pandas as pd
import json
from datetime import datetime, timezone
from mftool import Mftool
from tqdm import tqdm
master_funds_list = pd.read_csv("mf_app/data/scheme_master_full.csv")
master_funds_list

,scheme_code,scheme_name,amc,plan,risk_profile,created_at,started_date,eq_or_dt,type_of_mf
0,119550,Aditya Birla Sun Life Banking & PSU Debt Fund-...,Aditya Birla Sun Life Mutual Fund,Direct,Very High,2026-02-09 09:23:27.556760+00:00,02-01-2013,Debt Scheme,Banking and PSU Fund
1,120438,Axis Banking & PSU Debt Fund - Direct Plan - G...,Axis Mutual Fund,Direct,Very High,2026-02-09 09:23:28.046142+00:00,02-01-2013,Debt Scheme,Banking and PSU Fund
2,152162,Bajaj Finserv Banking and PSU Fund-Direct Plan...,Bajaj Finserv Mutual Fund,Direct,Very High,2026-02-09 09:23:28.397393+00:00,15-11-2023,Debt Scheme,Banking and PSU Fund
3,121279,Bandhan Banking and PSU Fund - Direct Growth,Bandhan Mutual Fund,Direct,Very High,2026-02-09 09:23:28.923424+00:00,08-03-2013,Debt Scheme,Banking and PSU Fund
4,150505,Canara Robeco Banking and PSU Debt Fund- Direc...,Canara Robeco Mutual Fund,Direct,Very High,2026-02-09 09:23:29.299342+00:00,24-08-2022,Debt Scheme,Banking and PSU Fund
...,...,...,...,...,...,...,...,...,...
2512,151341,UTI Fixed Term Income Fund - Series XXXV-III (...,UTI Mutual Fund,Direct,Very High,2026-02-09 09:43:55.092747+00:00,30-01-2023,Other,IDF
2513,151485,UTI Fixed Term Income Fund Series XXXVI - I (1...,UTI Mutual Fund,Direct,Very High,2026-02-09 09:43:55.500099+00:00,03-03-2023,Other,IDF
2514,122165,UTI Quarterly Interval Fund - I -Direct Plan -...,UTI Mutual Fund,Direct,Very High,2026-02-09 09:43:56.130233+00:00,28-03-2013,Other,IDF
2515,121182,UTI Quarterly Interval Fund - II - Direct Plan...,UTI Mutual Fund,Direct,Very High,2026-02-09 09:43:56.587891+00:00,08-02-2013,Other,IDF


In [22]:
from datetime import datetime
url = "http://localhost:8000/funds"
input_format = "%d-%m-%Y"
output_format = "%Y-%m-%d"

for row in tqdm(master_funds_list.iterrows()):
    req_payload = {
        'scheme_code': str(row[1]['scheme_code']),
        'scheme_name': row[1]['scheme_name'],
        'amc': row[1]['amc'],
        'plan': row[1]['plan'],
        'risk_profile': row[1]['risk_profile'],
        'started_date': datetime.strptime(row[1]['started_date'],input_format).strftime(output_format),
        'eq_or_dt': row[1]['eq_or_dt'],
        'type_of_mf': row[1]['type_of_mf']
    }

    response = requests.post(url, json=req_payload)
    # Check the response status code
    if response.status_code == 200 or response.status_code == 201:
        print("Request successful!")
        # Parse the JSON response into a Python dictionary
        returned_data = response.json()
        print(json.dumps(returned_data, indent=4)) # Pretty print the result
    else:
        print(f"Request failed with status code: {response.status_code}")
        print(response.text)



58it [00:00, 295.38it/s]

Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request 

125it [00:00, 320.92it/s]

Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request failed with status code: 400
{"detail":"Fund exists"}
Request 

195it [00:00, 334.40it/s]

...

Request successful!
{
    "scheme_code": "122642",
    "scheme_name": "UTI Fixed  Income Interval Fund ( Half Yearly Plan - I) - Direct Plan - Growth Option",
    "amc": "UTI Mutual Fund",
    "plan": "Direct",
    "risk_profile": "Very High",
    "started_date": "2013-05-29T00:00:00",
    "eq_or_dt": "Other",
    "type_of_mf": "IDF",
    "id": 2415,
    "created_at": "2026-02-09T10:42:11.033338"
}
Request successful!
{
    "scheme_code": "125953",
    "scheme_name": "UTI Fixed Income Interval Fund ( Half Yearly Plan-II)- Direct Plan - Growth Option",
    "amc": "UTI Mutual Fund",
    "plan": "Direct",
    "risk_profile": "Very High",
    "started_date": "2013-12-05T00:00:00",
    "eq_or_dt": "Other",
    "type_of_mf": "IDF",
    "id": 2416,
    "created_at": "2026-02-09T10:42:11.037407"
}
Request successful!
{
    "scheme_code": "120970",
    "scheme_name": "UTI - Fixed Income Interval Fund - Monthly Interval Plan  - Direct Plan - Growth Option",
    "amc": "UTI Mutual Fund",
    "pla

2490it [00:09, 221.06it/s]

Request successful!
{
    "scheme_code": "146608",
    "scheme_name": "UTI - Fixed Term Income Fund Series XXXI-VII (1155 Days) - Direct Plan - Growth Option",
    "amc": "UTI Mutual Fund",
    "plan": "Direct",
    "risk_profile": "Very High",
    "started_date": "2019-03-15T00:00:00",
    "eq_or_dt": "Other",
    "type_of_mf": "IDF",
    "id": 2456,
    "created_at": "2026-02-09T10:42:11.238539"
}
Request successful!
{
    "scheme_code": "146706",
    "scheme_name": "UTI - Fixed Term Income Fund Series XXXI-VIII (1153 Days) - Direct Plan - Growth Option",
    "amc": "UTI Mutual Fund",
    "plan": "Direct",
    "risk_profile": "Very High",
    "started_date": "2019-03-11T00:00:00",
    "eq_or_dt": "Other",
    "type_of_mf": "IDF",
    "id": 2457,
    "created_at": "2026-02-09T10:42:11.242614"
}
Request successful!
{
    "scheme_code": "146872",
    "scheme_name": "UTI - Fixed Term Income Fund Series XXXI-X (1168 Days) - Direct Plan - Growth Option",
    "amc": "UTI Mutual Fund",
    "

2517it [00:09, 266.45it/s]

Request successful!
{
    "scheme_code": "142361",
    "scheme_name": "UTI FTIF Series XXVIII-VIII (1171 Days) - Direct Plan - Growth Option",
    "amc": "UTI Mutual Fund",
    "plan": "Direct",
    "risk_profile": "Very High",
    "started_date": "2018-02-02T00:00:00",
    "eq_or_dt": "Other",
    "type_of_mf": "IDF",
    "id": 2503,
    "created_at": "2026-02-09T10:42:11.442545"
}
Request successful!
{
    "scheme_code": "142578",
    "scheme_name": "UTI FTIF Series XXVIII-X (1153 Days) - Direct Plan - Growth Option",
    "amc": "UTI Mutual Fund",
    "plan": "Direct",
    "risk_profile": "Very High",
    "started_date": "2018-03-14T00:00:00",
    "eq_or_dt": "Other",
    "type_of_mf": "IDF",
    "id": 2504,
    "created_at": "2026-02-09T10:42:11.446469"
}
Request successful!
{
    "scheme_code": "142789",
    "scheme_name": "UTI FTIF Series XXVIII-XI (1161 Days) - Direct Plan - Growth Option",
    "amc": "UTI Mutual Fund",
    "plan": "Direct",
    "risk_profile": "Very High",
    "